<a href="https://colab.research.google.com/github/JuhiNathani/CodeClauseInternship_MarketBasketAnalysis/blob/main/Code_Clause_Market_Basket_Analysis_Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Market Basket Analysis**

In [15]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
## Code to install any package via python

def install_and_import(package):
    import importlib
    try:
        importlib.import_module(package)
    except ImportError:
        import pip
        pip.main(['install', package])
    finally:
        globals()[package] = importlib.import_module(package)

install_and_import('mlxtend')

Installing packages:

In [17]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

Importing the data:

In [ ]:
df = pd.read_excel('Online Retail.xlsx')
df.head()

In [ ]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis = 0, subset=['InvoiceNo'], inplace = True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]

Before proceeding, let us understand the data distribution by country:

In [ ]:
df.groupby('Country').count().reset_index().sort_values('InvoiceNo', ascending = False).head()

In [ ]:
Basket = (df[df['Country']=="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

Basket.head()

In-order to complete the one-hot encoding process, we need to replace all values of quantity >=1 by 1.

In [ ]:
def sum_to_boolean(x):
    if x<=0:
        return 0
    else:
        return 1

Basket_Final = Basket.applymap(sum_to_boolean)


Dropping the postage column, and the final one-hot codded matrix.

In [ ]:
Basket_Final.drop('POSTAGE', inplace=True, axis=1)

Basket_Final.head()

** Apriori:**

To start with and have sufficient data, let us look at frequent itemsets that have a support of atleast 6%.

In [ ]:
## Apriori to select the most important itemsets
Frequent_itemsets = apriori(Basket_Final, min_support = 0.06, use_colnames = True)

Frequent_itemsets.sort_values('support', ascending = False).head()

** Association Rules:**

Now since we have identified the key itemsets, let us apply the association rules to learn the purchase behaviours.

In [ ]:
Asso_Rules = association_rules(Frequent_itemsets, metric = "lift", min_threshold = 1)
Asso_Rules.sort_values('lift',ascending = False)

In [ ]:
Basket['PLASTERS IN TIN WOODLAND ANIMALS'].sum()

In [ ]:
Basket['PLASTERS IN TIN CIRCUS PARADE'].sum()



** Purchase behaviour in UK:**



In [ ]:
Basket_France = (df[df['Country']=="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

def sum_to_boolean(x):
    if x<=0:
        return 0
    else:
        return 1

Basket_Final_France = Basket_France.applymap(sum_to_boolean)
Frequent_itemsets_France = apriori(Basket_Final_France, min_support = 0.06, use_colnames = True)

Asso_Rules_France = association_rules(Frequent_itemsets_France, metric = "lift", min_threshold = 1)
Asso_Rules_France.sort_values('lift',ascending = False).head()

Thus, it is interesting see that the most purchased together items in France are Cutlery Dolly Girls and Spaceboy, Red and green alarms and so on.